In [4]:
import uproot3 as uproot
from uproot3_methods import TLorentzVectorArray
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import scipy.stats
import pyhf # limit computations
import pyhf.contrib.viz.brazil
pyhf.set_backend("numpy")
from scipy.optimize import toms748
from tqdm import tqdm

In [5]:
def get_p4(tree, key):
    """
    Retrieve a 4-vector object from ROOT using the fCoordinates properties
    
    Necessary when reading ROOT files in via Uproot, since Uproot does not natively 
    recognize and convert LorentzVector objects
    """
    return TLorentzVectorArray.from_ptetaphim(
        tree[key]["fCoordinates.fPt"].array(),
        tree[key]["fCoordinates.fEta"].array(),
        tree[key]["fCoordinates.fPhi"].array(),
        tree[key]["fCoordinates.fM"].array(),
    )

In [6]:
def get_df(signal="vbshwwlvlvbb_c2v", lhe_rewgt_idx=-1):
    samples = [
        "tt1lpowheg", "tt2lpowheg", "ttw", "ttz", "raretop",
        "bosons", signal
    ]
    baby_dir = "/nfs-7/userdata/jguiang/VBSHWWBaby/v2.6_SS/v2/createMini_Run2"
    df = []
    for sample_path in glob.glob(f"{baby_dir}/*.root"):
        name = sample_path.split("/")[-1].split(".root")[0]
        if name not in samples:
            continue
        # Retrieve TTree
        with uproot.open(sample_path) as f:
            tree = f.get("variable")
            temp_df = tree.pandas.df(
                branches=["is_ps*", "wgt", "btagsf", "lepsf", 
                          "xsec_sf", "genrewgt", "trigsf", "pu_rewgt",
                          "drbb", "mbb", "mbbIn", "ptbb", 
                          "mjj", "detajj", "lt", "st"]
            )
            # Read in useful 4-vectors
            ld_lep_p4 = get_p4(tree, "leadlep")
            tr_lep_p4 = get_p4(tree, "subllep")
            # Set simple columns from 4-vectors
            temp_df["ld_lep_pt"] = ld_lep_p4.pt
            temp_df["tr_lep_pt"] = tr_lep_p4.pt
            # Compute additional columns
            temp_df["name"] = name
            temp_df["is_signal"] = (name == signal)
            temp_df["weight"] = (temp_df.wgt
                                 * temp_df.btagsf
                                 * temp_df.lepsf
                                 * temp_df.genrewgt
                                 * temp_df.trigsf
                                 * temp_df.pu_rewgt)
            # Apply sig-specific event weights
            if name == signal and lhe_rewgt_idx >= 0:
                temp_df["weight"] *= tree["lherewgts"].array()[:,lhe_rewgt_idx]
            # Apply bkg-specific event weights
            if name != signal:
                temp_df["weight"] *= temp_df.xsec_sf
            # Drop the columns we don't need
            temp_df.drop(
                columns=["wgt", "btagsf", "lepsf", "xsec_sf", "genrewgt", 
                         "trigsf", "pu_rewgt"], 
                inplace=True
            )
            df.append(temp_df)

    # Put dataframe together
    df = pd.concat(df)
    # Cast boolean-like columns to proper bools
    bool_like_cols = df.columns[df.columns.str.contains("is_")]
    df[bool_like_cols] = df[bool_like_cols].astype(bool)
    # Make sample col a categorical
    df["name"] = df.name.astype("category")
    return df

In [7]:
def upperlimit_auto(data, model, low, high, level=0.05, atol=2e-12, rtol=1e-15,
                    obs_eq_exp=False):
    """
    Calculate an upper limit interval ``(0, poi_up)`` for a single
    Parameter of Interest (POI) using an automatic scan through
    POI-space, using the TOMS748 algorithm.

    ..., mostly copied from upperlimit docstring.
    """

    def f_all(mu):
        if high > 10:
            par_bounds=[(b[0], high) for b in model.config.suggested_bounds()]
        else:
            par_bounds=None
        return pyhf.infer.hypotest(
            mu, data, model, test_stat="qtilde", return_expected_set=True, par_bounds=par_bounds
        )

    def f(mu, limit=0):
        # Use integers for limit so we don't need a string comparison
        if limit == 0:
            # Obs
            return f_all(mu)[0] - level
        else:
            # Exp (These are in the order -2, -1, 0, 1, 2 sigma)
            return f_all(mu)[1][limit - 1] - level

    tb, _ = pyhf.get_backend()
    obs = tb.astensor(toms748(f, low, high, args=(0), k=2, xtol=atol, rtol=rtol))
    if obs_eq_exp:
        return float(obs)
    else:
        exp = [
            tb.astensor(toms748(f, low, high, args=(i), k=2, xtol=atol, rtol=rtol))
            for i in range(1, 6)
        ]
        return float(obs), exp

def figure_of_merit(sig_counts, bkg_counts, bkg_errors=[], obs_counts=[], 
                    low=0.1, high=10000, tol=0.001):
    sig_counts = [S if S > 0 else 1e-6 for S in sig_counts]
    bkg_counts = [B if B > 0 else 1e-6 for B in bkg_counts]
    if len(bkg_errors) == 0:
        bkg_errors = [0.6*B for B in bkg_counts]
    model = pyhf.simplemodels.hepdata_like(
        signal_data=sig_counts,
        bkg_data=bkg_counts,
        bkg_uncerts=bkg_errors
    )
    if len(obs_counts) == 0:
        obs_counts = bkg_counts
    data = obs_counts + model.config.auxdata
    try:
        return upperlimit_auto(
            data, model,
            low, high,
            level=0.05, atol=tol, rtol=tol,
            obs_eq_exp=True
        )
    except ValueError as e:
        print(f"WARNING: {e}")
        print(f"Defaulting to error-handling behaviour: returning {high}")
        return high

In [8]:
def grand_table(limit_low=1, limit_high=1000, *args, **kwargs):
    df = get_df(*args, **kwargs)
    # Algorithmically chosen and rounded and outside of other SRs plus original BSM SRs
    c2v_sr = "detajj > 5 and lt >= 300 and st >= 700"
    c3_sr = f"lt >= 200.0 and st >= 400.0 and detajj > 5" # (AND Mbb WINDOW IMPLICITLY!!)
    sm_sr = f"mjj >= 1700 and detajj > 5" # (AND Mbb WINDOW IMPLICITLY!!)

    rows = []
    for name, channel in STRAWMEN(df):
        in_sm_sr = df.eval(f"{sm_sr} and not ({c3_sr}) and not ({c2v_sr})")
        sig_counts = df[channel & in_sm_sr &  df.is_signal].weight.values
        bkg_counts = df[channel & in_sm_sr & ~df.is_signal].weight.values
        rows.append({
            "target": "SM",
            "channel": name,
            "region": f"{sm_sr} and not ({c3_sr}) and not ({c2v_sr})",
            "sig": sig_counts.sum(),
            "sig_err": np.sqrt(np.sum(sig_counts**2)),
            "bkg": bkg_counts.sum(),
            "bkg_err": np.sqrt(np.sum(bkg_counts**2)),
            "limit": figure_of_merit([sig_counts.sum()], [bkg_counts.sum()], low=limit_low, high=limit_high)
        })

    for name, channel in STRAWMEN(df):
        in_c3_sr = df.eval(f"{c3_sr} and not ({c2v_sr})")
        sig_counts = df[channel & in_c3_sr &  df.is_signal].weight.values
        bkg_counts = df[channel & in_c3_sr & ~df.is_signal].weight.values
        rows.append({
            "target": "C3",
            "channel": name,
            "region": f"{c3_sr} and not ({c2v_sr})",
            "sig": sig_counts.sum(),
            "sig_err": np.sqrt(np.sum(sig_counts**2)),
            "bkg": bkg_counts.sum(),
            "bkg_err": np.sqrt(np.sum(bkg_counts**2)),
            "limit": figure_of_merit([sig_counts.sum()], [bkg_counts.sum()], low=limit_low, high=limit_high)
        })

    base_c2v_sr = "detajj >= 5"
    tight_c2v_sr = "lt >= 500 and st >= 900"
    loose_c2v_sr = f"lt >= 300 and st >= 700 and not ({tight_c2v_sr})"
    for name, channel in STRAWMEN(df):
        in_c2v_sr = df.eval(f"{base_c2v_sr} and {tight_c2v_sr}")
        sig_counts = df[channel & in_c2v_sr &  df.is_signal].weight.values
        bkg_counts = df[channel & in_c2v_sr & ~df.is_signal].weight.values
        rows.append({
            "target": "C2V/CV",
            "channel": name,
            "region": f"{base_c2v_sr} and {tight_c2v_sr}",
            "sig": sig_counts.sum(),
            "sig_err": np.sqrt(np.sum(sig_counts**2)),
            "bkg": bkg_counts.sum(),
            "bkg_err": np.sqrt(np.sum(bkg_counts**2)),
            "limit": figure_of_merit([sig_counts.sum()], [bkg_counts.sum()], low=limit_low, high=limit_high)
        })
    in_c2v_sr = df.eval(f"{base_c2v_sr} and {loose_c2v_sr}")
    sig_counts = df[STRAWMAN_LEP_LEP(df) & in_c2v_sr &  df.is_signal].weight.values
    bkg_counts = df[STRAWMAN_LEP_LEP(df) & in_c2v_sr & ~df.is_signal].weight.values
    rows.append({
        "target": "C2V/CV",
        "channel": name,
        "region": f"{base_c2v_sr} and {loose_c2v_sr}",
        "sig": sig_counts.sum(),
        "sig_err": np.sqrt(np.sum(sig_counts**2)),
        "bkg": bkg_counts.sum(),
        "bkg_err": np.sqrt(np.sum(bkg_counts**2)),
        "limit": figure_of_merit([sig_counts.sum()], [bkg_counts.sum()], low=limit_low, high=limit_high)
    })

    return pd.DataFrame(data=rows)

def grand_table_tex(table, sig_count_round=3, bkg_count_round=3, sig_error_round=3, bkg_error_round=3,
                    limit_round=3, limit_low=1, limit_high=1000):
    textable = table.copy()
    textable["Signal Region"] = [
        "$\muchan$", "$\elchan$", "$\tauchan$", "$\negchan$",
        "$\muchan$", "$\elchan$", "$\tauchan$", "$\negchan$",
        "$\muchan$", "$\elchan$", "$\tauchan$", "$\negchan$", "$\lgtchan$",
    ]
    sig_counts = textable.sig.round(sig_count_round)
    sig_errors = textable.sig_err.round(sig_error_round)
    textable["$N_{sig}$"] = sig_counts.astype(str)+" $\pm$ "+sig_errors.astype(str)
    bkg_counts = textable.bkg.round(bkg_count_round)
    bkg_errors = textable.bkg_err.round(bkg_error_round)
    textable["$N_{bkg}$"] = bkg_counts.astype(str)+" $\pm$ "+bkg_errors.astype(str)
    textable["$\mu_{0.05}$"] = textable.limit.round(limit_round)
    textable.drop(columns=table.columns, inplace=True)
    textable = textable.reindex([1,0,2,3,5,4,6,7,9,8,10,11,12])
    # Generate LaTeX
    tex = textable.to_latex(index=False, escape=False)
    # Insert signal region labels
    tex = tex.split("\n")
    tex.insert(4, "Standard Model \\\\")
    tex.insert(9, "BSM \CTHREE\\\\")
    tex.insert(14, "BSM \CTWOV/\CV\\\\")
    # Insert combined limit
    limit = figure_of_merit(table.sig.values, table.bkg.values, low=limit_low, high=limit_high)
    tex.insert(-3, "\\midrule")
    tex.insert(-3, f"Combined & & & {round(limit, limit_round)} \\\\")
    # Put everthing back together
    tex = "\n".join(tex)
    print(tex)

In [9]:
BASE_SEL = lambda df: (df.is_ps & df.mbbIn)
STRAWMAN_LEAD_EL = lambda df: (BASE_SEL(df) & df.is_ps_el)
STRAWMAN_LEAD_MU = lambda df: (BASE_SEL(df) & df.is_ps_mu)
STRAWMAN_LEP_TAU = lambda df: (BASE_SEL(df) & df.is_ps_tau)
STRAWMAN_NEG_NEG = lambda df: (BASE_SEL(df) & df.is_ps_neg)
STRAWMEN = lambda df: [
    ("mu+l+", STRAWMAN_LEAD_MU(df)),
    ("e+l+", STRAWMAN_LEAD_EL(df)),
    ("l+tau+", STRAWMAN_LEP_TAU(df)),
    ("lep-lep-", STRAWMAN_NEG_NEG(df))
]
STRAWMAN_LEP_LEP = lambda df: (BASE_SEL(df) & df.is_ps_lgt)

In [10]:
table = grand_table(lhe_rewgt_idx=20, limit_low=10, limit_high=90000)
display(table)
print(table.to_csv(index=False))
grand_table_tex(
    table, 
    sig_count_round=4,
    sig_error_round=5,
    bkg_count_round=3,
    bkg_error_round=3,
    limit_round=0
)

/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(


,target,channel,region,sig,sig_err,bkg,bkg_err,limit
0,SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.004161,0.001399,0.352980,0.084058,648.425704
1,SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.001178,0.000586,0.255398,0.083628,2157.780034
2,SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.000071,0.000035,1.257972,0.228048,53487.005787
3,SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.001380,0.000746,1.394870,0.248908,2857.331427
4,C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.009736,0.001605,0.966229,0.137156,357.688999
5,C3,e+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.004908,0.000962,0.644597,0.158601,631.632703
6,C3,l+tau+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.000727,0.000216,2.780226,0.328015,7251.052263
7,C3,lep-lep-,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.003966,0.001088,3.776705,0.349602,1552.030677
8,C2V/CV,mu+l+,detajj >= 5 and lt >= 500 and st >= 900,0.001266,0.000063,0.053186,0.032340,1672.868970
9,C2V/CV,e+l+,detajj >= 5 and lt >= 500 and st >= 900,0.000671,0.000053,0.003350,0.009005,2892.965086


target,channel,region,sig,sig_err,bkg,bkg_err,limit
SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.0041614794,0.0013994324,0.3529801,0.08405838,648.4257042608524
SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.0011776742,0.0005859799,0.2553982,0.083627984,2157.780034312986
SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),7.105085e-05,3.4754994e-05,1.2579715,0.22804825,53487.005786573565
SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.0013800063,0.000746082,1.3948704,0.24890815,2857.3314266886555
C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and not (detajj > 5 and lt >= 300 and st >= 700),0.009736216,0.0016051346,0.96

/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(


\begin{tabular}{lllr}
\toprule
Signal Region &            $N_{sig}$ &         $N_{bkg}$ &  $\mu_{0.05}$ \\
\midrule
Standard Model \\
    $\elchan$ & 0.0012 $\pm$ 0.00059 & 0.255 $\pm$ 0.084 &        2158.0 \\
    $\muchan$ &  0.0042 $\pm$ 0.0014 & 0.353 $\pm$ 0.084 &         648.0 \\
   $\tauchan$ &    1e-04 $\pm$ 3e-05 & 1.258 $\pm$ 0.228 &       53487.0 \\
   $\negchan$ & 0.0014 $\pm$ 0.00075 & 1.395 $\pm$ 0.249 &        2857.0 \\
BSM \CTHREE\\
    $\elchan$ & 0.0049 $\pm$ 0.00096 & 0.645 $\pm$ 0.159 &         632.0 \\
    $\muchan$ & 0.0097 $\pm$ 0.00161 & 0.966 $\pm$ 0.137 &         358.0 \\
   $\tauchan$ & 0.0007 $\pm$ 0.00022 &  2.78 $\pm$ 0.328 &        7251.0 \\
   $\negchan$ &  0.004 $\pm$ 0.00109 &  3.777 $\pm$ 0.35 &        1552.0 \\
BSM \CTWOV/\CV\\
    $\elchan$ &   0.0007 $\pm$ 5e-05 & 0.003 $\pm$ 0.009 &        2893.0 \\
    $\muchan$ &   0.0013 $\pm$ 6e-05 & 0.053 $\pm$ 0.032 &        1673.0 \\
   $\tauchan$ &   0.0003 $\pm$ 2e-05 & 0.007 $\pm$ 0.003 &        6279.0 \\

In [11]:
table = grand_table(lhe_rewgt_idx=-1, limit_low=1, limit_high=10000)
display(table)
print(table.to_csv(index=False))
grand_table_tex(table, limit_round=2, limit_low=0.1, limit_high=10)

/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(


,target,channel,region,sig,sig_err,bkg,bkg_err,limit
0,SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.028277,0.004964,0.352980,0.084058,95.426059
1,SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.009538,0.002875,0.255398,0.083628,266.441757
2,SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.005951,0.002268,1.257972,0.228048,638.582945
3,SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.009082,0.002919,1.394870,0.248908,434.178721
4,C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.436924,0.020764,0.966229,0.137156,7.970473
5,C3,e+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.229612,0.013993,0.644597,0.158601,13.501676
6,C3,l+tau+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.111197,0.009439,2.780226,0.328015,47.388289
7,C3,lep-lep-,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.162880,0.012193,3.776705,0.349602,37.789604
8,C2V/CV,mu+l+,detajj >= 5 and lt >= 500 and st >= 900,1.658314,0.040238,0.053186,0.032340,1.277610
9,C2V/CV,e+l+,detajj >= 5 and lt >= 500 and st >= 900,0.863704,0.027407,0.003350,0.009005,2.248515


target,channel,region,sig,sig_err,bkg,bkg_err,limit
SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.028277243,0.0049639097,0.3529801,0.08405838,95.42605932033914
SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.009538051,0.0028753183,0.2553982,0.083627984,266.441757369469
SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.0059511615,0.002267723,1.2579715,0.22804825,638.5829451308218
SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.009081887,0.002918941,1.3948704,0.24890815,434.17872065193023
C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and not (detajj > 5 and lt >= 300 and st >= 700),0.43692395,0.02076421,0.96622926,0.1

/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(


\begin{tabular}{lllr}
\toprule
Signal Region &         $N_{sig}$ &         $N_{bkg}$ &  $\mu_{0.05}$ \\
\midrule
Standard Model \\
    $\elchan$ &  0.01 $\pm$ 0.003 & 0.255 $\pm$ 0.084 &        266.44 \\
    $\muchan$ & 0.028 $\pm$ 0.005 & 0.353 $\pm$ 0.084 &         95.43 \\
   $\tauchan$ & 0.006 $\pm$ 0.002 & 1.258 $\pm$ 0.228 &        638.58 \\
   $\negchan$ & 0.009 $\pm$ 0.003 & 1.395 $\pm$ 0.249 &        434.18 \\
BSM \CTHREE\\
    $\elchan$ &  0.23 $\pm$ 0.014 & 0.645 $\pm$ 0.159 &         13.50 \\
    $\muchan$ & 0.437 $\pm$ 0.021 & 0.966 $\pm$ 0.137 &          7.97 \\
   $\tauchan$ & 0.111 $\pm$ 0.009 &  2.78 $\pm$ 0.328 &         47.39 \\
   $\negchan$ & 0.163 $\pm$ 0.012 &  3.777 $\pm$ 0.35 &         37.79 \\
BSM \CTWOV/\CV\\
    $\elchan$ & 0.864 $\pm$ 0.027 & 0.003 $\pm$ 0.009 &          2.25 \\
    $\muchan$ &  1.658 $\pm$ 0.04 & 0.053 $\pm$ 0.032 &          1.28 \\
   $\tauchan$ & 0.437 $\pm$ 0.019 & 0.007 $\pm$ 0.003 &          4.48 \\
   $\negchan$ &  0.424 $\pm$ 0.02 &

In [12]:
table = grand_table(signal="vbshwwlvlvbb_c3", lhe_rewgt_idx=-1, limit_low=1, limit_high=10000)
display(table)
print(table.to_csv(index=False))
grand_table_tex(table, limit_round=2, limit_low=0.1, limit_high=10.)

/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(


,target,channel,region,sig,sig_err,bkg,bkg_err,limit
0,SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.093564,0.012518,0.352980,0.084058,28.839849
1,SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.026780,0.006068,0.255398,0.083628,94.891716
2,SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.016265,0.004754,1.257972,0.228048,233.656250
3,SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.029098,0.006803,1.394870,0.248908,135.522311
4,C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.577872,0.030891,0.966229,0.137156,6.026426
5,C3,e+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.345024,0.022667,0.644597,0.158601,8.985330
6,C3,l+tau+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.141013,0.014288,2.780226,0.328015,37.368305
7,C3,lep-lep-,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.255212,0.019894,3.776705,0.349602,24.117840
8,C2V/CV,mu+l+,detajj >= 5 and lt >= 500 and st >= 900,0.205124,0.018615,0.053186,0.032340,10.327914
9,C2V/CV,e+l+,detajj >= 5 and lt >= 500 and st >= 900,0.114149,0.013108,0.003350,0.009005,17.013303


target,channel,region,sig,sig_err,bkg,bkg_err,limit
SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.093564495,0.01251809,0.3529801,0.08405838,28.83984868130478
SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.026779568,0.0060682776,0.2553982,0.083627984,94.89171568468416
SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.016264899,0.0047539016,1.2579715,0.22804825,233.65625024203626
SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200.0 and st >= 400.0 and detajj > 5) and not (detajj > 5 and lt >= 300 and st >= 700),0.029097894,0.0068030325,1.3948704,0.24890815,135.5223112025944
C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and not (detajj > 5 and lt >= 300 and st >= 700),0.57787216,0.03089106,0.96622926,0.1

/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(


\begin{tabular}{lllr}
\toprule
Signal Region &         $N_{sig}$ &         $N_{bkg}$ &  $\mu_{0.05}$ \\
\midrule
Standard Model \\
    $\elchan$ & 0.027 $\pm$ 0.006 & 0.255 $\pm$ 0.084 &         94.89 \\
    $\muchan$ & 0.094 $\pm$ 0.013 & 0.353 $\pm$ 0.084 &         28.84 \\
   $\tauchan$ & 0.016 $\pm$ 0.005 & 1.258 $\pm$ 0.228 &        233.66 \\
   $\negchan$ & 0.029 $\pm$ 0.007 & 1.395 $\pm$ 0.249 &        135.52 \\
BSM \CTHREE\\
    $\elchan$ & 0.345 $\pm$ 0.023 & 0.645 $\pm$ 0.159 &          8.99 \\
    $\muchan$ & 0.578 $\pm$ 0.031 & 0.966 $\pm$ 0.137 &          6.03 \\
   $\tauchan$ & 0.141 $\pm$ 0.014 &  2.78 $\pm$ 0.328 &         37.37 \\
   $\negchan$ &  0.255 $\pm$ 0.02 &  3.777 $\pm$ 0.35 &         24.12 \\
BSM \CTWOV/\CV\\
    $\elchan$ & 0.114 $\pm$ 0.013 & 0.003 $\pm$ 0.009 &         17.01 \\
    $\muchan$ & 0.205 $\pm$ 0.019 & 0.053 $\pm$ 0.032 &         10.33 \\
   $\tauchan$ & 0.057 $\pm$ 0.009 & 0.007 $\pm$ 0.003 &         34.33 \\
   $\negchan$ & 0.051 $\pm$ 0.009 &

In [13]:
def get_fluctuation(mean, target_prob, return_prob=False, _result=1):
    prob = scipy.stats.poisson.pmf(_result, mean)
    if prob > target_prob:
        return get_fluctuation(mean, target_prob, return_prob=return_prob, _result=_result+1)
    else:
        if return_prob:
            return prob
        else:
            return _result

In [14]:
# Generate table
print("Generating table...")
table = grand_table(lhe_rewgt_idx=-1, limit_low=1, limit_high=1000)
table["obs"] = table.bkg # "Observed" events = background for bkg-only hypothesis

# Compute "observed" events w/ < 5% probable fluctuations
table["obs_up1"] = table.bkg.apply(lambda x: get_fluctuation(x, 0.0505))
table["obs_up1_prob"] = table.bkg.apply(lambda x: get_fluctuation(x, 0.0505, return_prob=True))
# Compute limits
obs_up1_limits = []
for i in tqdm(table.index, desc="Computing up1 limits"):
    obs_counts = list(table.obs)
    obs_counts[i] = table.loc[i].obs_up1
    mu = figure_of_merit(
        table.sig.values, table.bkg.values, 
        obs_counts=obs_counts, 
        low=0.1, high=10, tol=0.001
    )
    obs_up1_limits.append(mu)
table["obs_up1_limit"] = obs_up1_limits

# Compute "observed" events w/ < 1% probable fluctuations
table["obs_up2"] = table.bkg.apply(lambda x: get_fluctuation(x, 0.01))
table["obs_up2_prob"] = table.bkg.apply(lambda x: get_fluctuation(x, 0.01, return_prob=True))
# Compute limits
obs_up2_limits = []
for i in tqdm(table.index, desc="Computing up2 limits"):
    obs_counts = list(table.obs)
    obs_counts[i] = table.loc[i].obs_up2
    mu = figure_of_merit(
        table.sig.values, table.bkg.values, 
        obs_counts=obs_counts, 
        low=0.1, high=10, tol=0.001
    )
    obs_up2_limits.append(mu)
table["obs_up2_limit"] = obs_up2_limits

display(table)

Generating table...


/tmp/ipykernel_2296663/1073179542.py:46: DeprecationWarning: pyhf.simplemodels.hepdata_like is deprecated in favor of pyhf.simplemodels.uncorrelated_background as of pyhf v0.6.2 and will be removed in release 0.7.0. Please use pyhf.simplemodels.uncorrelated_background.
  model = pyhf.simplemodels.hepdata_like(
Computing up2 limits: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:07<00:00,  5.23s/it]


,target,channel,region,sig,sig_err,bkg,bkg_err,limit,obs,obs_up1,obs_up1_prob,obs_up1_limit,obs_up2,obs_up2_prob,obs_up2_limit
0,SM,mu+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.028277,0.004964,0.352980,0.084058,95.426558,0.352980,2,0.043770,0.589423,3,0.005150,0.592084
1,SM,e+l+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.009538,0.002875,0.255398,0.083628,266.441768,0.255398,2,0.025263,0.585439,3,0.002151,0.586610
2,SM,l+tau+,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.005951,0.002268,1.257972,0.228048,638.586099,1.257972,4,0.029658,0.582147,5,0.007462,0.582248
3,SM,lep-lep-,mjj >= 1700 and detajj > 5 and not (lt >= 200....,0.009082,0.002919,1.394870,0.248908,434.178723,1.394870,4,0.039097,0.582312,6,0.002536,0.582568
4,C3,mu+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.436924,0.020764,0.966229,0.137156,7.970473,0.966229,4,0.013819,0.649044,5,0.002670,0.662379
5,C3,e+l+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.229612,0.013993,0.644597,0.158601,13.501680,0.644597,3,0.023430,0.626637,4,0.003776,0.638257
6,C3,l+tau+,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.111197,0.009439,2.780226,0.328015,47.388289,2.780226,6,0.039784,0.585783,8,0.005491,0.587137
7,C3,lep-lep-,lt >= 200.0 and st >= 400.0 and detajj > 5 and...,0.162880,0.012193,3.776705,0.349602,37.789717,3.776705,7,0.049792,0.585666,9,0.009864,0.587082
8,C2V/CV,mu+l+,detajj >= 5 and lt >= 500 and st >= 900,1.658314,0.040238,0.053186,0.032340,1.277502,0.053186,1,0.050431,0.966554,2,0.001341,1.314140
9,C2V/CV,e+l+,detajj >= 5 and lt >= 500 and st >= 900,0.863704,0.027407,0.003350,0.009005,2.248515,0.003350,1,0.003339,1.003677,1,0.003339,1.003677


In [15]:
channels = [
    "$\\muchan$", "$\\elchan$", "$\\tauchan$", "$\\negchan$",
    "$\\muchan$", "$\\elchan$", "$\\tauchan$", "$\\negchan$",
    "$\\muchan$", "$\\elchan$", "$\\tauchan$", "$\\negchan$", "$\\lgtchan$",
]
rows = []
for i in table.index:
    table_row = table.loc[i]
    rows.append({
        "Channel": channels[i],
        # orig
        "$\\langle N_{obs}\\rangle$": table_row.obs.round(3),
        # up1 col
        "$N_{obs}^{\\uparrow}$": table_row.obs_up1.round(3),
        "P($N_{obs}$ = $N_{obs}^{\\uparrow}$)": str((table_row.obs_up1_prob*100).round(1)) + "\%",
        "$\mu_{0.05}^{\\uparrow}$": table_row.obs_up1_limit.round(2),
        # up2 col
        "$N_{obs}^{\\uparrow\\uparrow}$": table_row.obs_up2.round(3),
        "P($N_{obs}$ = $N_{obs}^{\\uparrow\\uparrow}$)": str((table_row.obs_up2_prob*100).round(2)) + "\%",
        "$\mu_{0.05}^{\\uparrow\\uparrow}$": table_row.obs_up2_limit.round(2),
    })
textable = pd.DataFrame(data=rows)
# Generate LaTeX
tex = textable.to_latex(index=False, escape=False)
# Insert signal region labels
tex = tex.split("\n")
tex[0] = "\\begin{tabular}{rccccccc}"
tex.insert(4, "Standard Model \\\\")
tex.insert(9, "BSM \CTHREE \\\\")
tex.insert(14, "BSM \CTWOV/\CV \\\\")
# Put everthing back together
tex = "\n".join(tex)
print(tex)

\begin{tabular}{rccccccc}
\toprule
   Channel &  $\langle N_{obs}\rangle$ &  $N_{obs}^{\uparrow}$ & P($N_{obs}$ = $N_{obs}^{\uparrow}$) &  $\mu_{0.05}^{\uparrow}$ &  $N_{obs}^{\uparrow\uparrow}$ & P($N_{obs}$ = $N_{obs}^{\uparrow\uparrow}$) &  $\mu_{0.05}^{\uparrow\uparrow}$ \\
\midrule
Standard Model \\
 $\muchan$ &                     0.353 &                     2 &                               4.4\% &                     0.59 &                             3 &                                      0.51\% &                             0.59 \\
 $\elchan$ &                     0.255 &                     2 &                               2.5\% &                     0.59 &                             3 &                                      0.22\% &                             0.59 \\
$\tauchan$ &                     1.258 &                     4 &                               3.0\% &                     0.58 &                             5 &                                      0.75\% 

In [27]:
print(2, 0.053186, scipy.stats.poisson.pmf(2, 0.053186)) # muchan
print(2, 0.003350, scipy.stats.poisson.pmf(2, 0.003350)) # elchan
print(2, 0.007102, scipy.stats.poisson.pmf(2, 0.007102)) # tauchan
print(3, 0.072579, scipy.stats.poisson.pmf(3, 0.072579)) # negchan
print(3, 0.097092, scipy.stats.poisson.pmf(3, 0.097092)) # lgtchan

2 0.053186 0.001341115792034516
2 0.00335 5.59248376349648e-06
2 0.007102 2.5040729732571703e-05
3 0.072579 5.9259915910929785e-05
3 0.097092 0.0001384307498678271
